## Final sprint:
At the end of the last sprint I indicated 2 problems that happered the full workability of this worksheet. 
1) stack overflow from python / data limits from jupyter.
2) problems with visualization.

1) I belief this problem is due to the inability of python to optimize their tail recursions. this means that after a certain limit their is a stackover from the language itself.
Combined with this I should have spread the load of the calculation over several functions instead of putting it all in one. 
2) The problem of visualization has not been clearly solved. But I belief I have found a work around. 

below you will find the reworked functions.

In [1]:
# importing pandas for the dataframes
import pandas as pd
import matplotlib.pyplot as plt
import math as math

#importing sys to increase the stack limit, YOU NEED TO RUN YOUR PYTHON CODE IN ROOT (MAC) OTHERWISE YOU GET 
#'NOT ALLOWED TO RAISE MAXIMUM LIMIT' VALUEERROR MESSAGE.
import resource
resource.setrlimit(resource.RLIMIT_STACK, (2**29,-1))

#importing the higher function reduce for a specific part of the code
from functools import reduce


#the proces ID to get into bigquery
projectid = "vast-art-167309"

In [2]:

class acceleration:    
                               
    def bigquery_adjusted (self, vehicle):
        """specific function which targets the bigquery database and reformats it"""
        car_list = []
        #the query to bigquery
        query_df = pd.read_gbq("select timestamp, value, vehicle from [headingon_params.speed_obd] where vehicle = '%s' order by timestamp asc" %(vehicle), projectid)
        end_row = query_df['timestamp'].count() -1
        return self.filter_function_forloop(vehicle, end_row, query_df, car_list)

    def filter_function_forloop (self,vehicle, end, dataframe, car_list):
        """This function filters the data which is pulled by the f(bigquery_adjusted) and filters out the blocks that will be used
            by f(core_function_forloop) """
        for row in range (0, end):
            if dataframe.iloc[row,1] == 0.0:
                row_add = row + 1
                if dataframe.iloc[row_add,1] != 0.0:
                    row_2_added = row_add + 1
                    if dataframe.iloc[row_2_added,1] != 0.0:
                        row_3_added = row_2_added + 1
                        if dataframe.iloc[row_3_added,1] != 0.0:
                            self.core_function_forloop(vehicle, row, dataframe, car_list)
                        else:
                            pass
                    else:
                        pass
                else:
                    pass
            else:
                pass
        return car_list
            
    def core_function_forloop(self,vehicle, row, dataframe, car_list):
        """this function calculates the acceleration over a 3 second window"""
        temp_list = []
        timestamp = dataframe.iat[row,0]
        for i in range(1,4):
            temp_row = row + i
            temp_list.append(dataframe.iat[temp_row,1])
        average_km = (reduce((lambda x, y: x+y), temp_list) / 3) 
        car_list.append({'timestamp': timestamp, 'value': average_km, 'vehicle': vehicle})
        return

###########################################################################################################################
####This part of the class acceleration is my coded version of the Local Outlier Factor ###################################
###########################################################################################################################
    
    
    def selecting_k_distance_function(self, dataframe, k, car):
        """Main function under which all other function for determining the LOF factor is placed"""
        #determines the length of the dataframe
        row_dataframe = dataframe['timestamp'].count()
        k_value = k + 1
        dataframe= dataframe.round(1)
        #counter which will be used as x coordinate
        dataframe['counter'] = range(len(dataframe))
        #unique id with x and y coordinates
        dataframe['unique_id'] = list(zip(dataframe['counter'], dataframe['value']))
        column_layout = ['unique_id']
        column_values = []
        observation_values = []
        #builds the appropriate dataframe size according to the inputs given
        for i in range(1, k_value):
            dataframe['value_k' + str(i)] = 1000
            column_layout.append('value_k' + str(i))
            column_values.append('value_k' + str(i))
            dataframe['observation_k' + str(i)] = -1
            column_layout.append('observation_k' + str(i))
            observation_values.append('observation_k' + str(i))
        dataframe = dataframe[column_layout]
        #calculating the Local reach density
        for i in range(1, k_value):
            self.all_columns_function(0,dataframe, i)
        dataframe['sum for LRD'] =  dataframe[column_values].sum(axis=1)
        dataframe['LRD'] = 1/(dataframe['sum for LRD']/k)
        
        observations =[]
        # not sure to be happy or worried about this fact that i can do a for-loop and listcomprehension simultaneously
        # reason for this choice: a list comprehension does not change data once '=' is reached, at least that is what my debugger insinuates 
        for i in dataframe[observation_values]:
            observations.append('check_'+ i)
            dataframe['check_'+ i] = [dataframe.loc[dataframe.loc[j,i], 'LRD'] for j in range(row_dataframe)]
        dataframe['LOF'] = (dataframe[observations].sum(axis=1)/k)/dataframe.loc[:,'LRD']
        
        inliers_dataframe = dataframe[dataframe['LOF'] < 1.6]
        outliers_dataframe = dataframe[dataframe['LOF'] > 1.5]
        
        inliers_dataframe['counter'], inliers_dataframe['value'] = zip(*inliers_dataframe['unique_id'])
        inliers_dataframe.plot.scatter(x='counter', y='value', title=str(car) + " WITHOUT outliers")
        
        inliers_dataframe.hist(column='value', normed=1)

    
    def all_columns_function(self, base_row,dataframe, k):
        """this function goes composes all the necessary columns"""
        row_dataframe = (dataframe['unique_id'].count() - 1) 
        if base_row > row_dataframe:
            return
        row_list_seq = list(range((base_row+1), (row_dataframe + 1)))
        row_list_seq.extend(range(base_row))
        for i in row_list_seq:
            self.min_function(base_row,i ,row_dataframe, dataframe, k)   
        base_row +=1
        return self.all_columns_function(base_row,dataframe, k)       
        
    def min_function(self,base_row, next_row,row_dataframe,dataframe, k):
        """This function together with f(euclides_function) determines the lowest euclidean value and places it in the correc
            column"""
        if k > 1:
            other_column_value = dataframe.loc[base_row,('value_k' + str(k-1))]
            current_value = dataframe.loc[base_row, ('value_k' + str(k))]
            temp_value = self.euclides_function(base_row,next_row, row_dataframe, dataframe)
            if (temp_value < current_value and temp_value > other_column_value):
                dataframe.loc[base_row,('value_k' + str(k))] = temp_value
                coordinate_of_value = dataframe.iloc[next_row, 0]
                coordinate_column = 'observation_k' + str(k)
                dataframe.set_value(base_row,coordinate_column,coordinate_of_value[0])
            else:
                pass
        else:
            current_value = dataframe.loc[base_row, ('value_k' + str(k))]
            temp_value = self.euclides_function(base_row,next_row, row_dataframe, dataframe)
            if temp_value < current_value:
                dataframe.loc[base_row,('value_k' + str(k))] = temp_value
                coordinate_of_value = dataframe.iloc[next_row, 0]
                coordinate_column = 'observation_k' + str(k)
                dataframe.set_value(base_row,coordinate_column,coordinate_of_value[0])
            else:
                pass      
    
    
      #the formula to calculate the euclidean distance between two point (as the crows fly)
    def euclides_function (self, base_row, next_row, row_dataframe,dataframe):
        """calculates either the euclidean value or the straight line depending on the circumstance"""
        tuple_base = dataframe.iloc[base_row,0]
        tuple_next = dataframe.iloc[next_row,0]  
        
        if tuple_base[1] == tuple_next[1]:
            euclide_distance = abs(tuple_base[0]-tuple_next[0])
            return euclide_distance
        else:
            euclide_distance = math.sqrt(math.pow((tuple_base[0] - tuple_next[0]),2) + 
                                         math.pow((tuple_base[1] - tuple_next[1]),2))
            return euclide_distance
        
        
def everything_together (list_vehicle, k_value):
    """retrieving the data from the bigquery, calculating the acceleration, with the k value"""
    for car in list_vehicle:
        car_present = acceleration()
        car_present.dataframeALL = pd.DataFrame(car_present.bigquery_adjusted(car))
        car_present.dataframeALL['counter'] = range(len(car_present.dataframeALL))
        car_present.dataframeALL.plot.scatter(x='counter', y='value', title=str(car) + " with outliers")
        car_present.dataframe_validation = car_present.dataframeALL[(car_present.dataframeALL['value'] < 1)]
        print(car_present.dataframe_validation)
        car_present.selecting_k_distance_function(car_present.dataframeALL,k_value, car)
    return   

### list of cars:
The above has been individually tested and works as expected. Bellow we have composed a new list of all the names of the cars. I am using the old names since the new names seem to have some problems in their unique identification keys.

also, we will use a k of 7. I have done several tests and the value of k has a significant influence on the outliers up until a point afterwhich it starts to plateau. However, as indictated in my previous sprint I have not found any clear writting on *which* k is best for the amount of observations being used. So I am going for k=7 though the program has been written in a way that you can change the k arbitrarily. Which I have done bellow for comvenience. But feel free to experiment.

In [3]:
#cars_list = [ 'car2', 'car4', 'car5']
cars_list = [ 'car1', 'car2', 'car3', 'car4', 'car5', 'car6', 'car7']

#cars_list=['4T3BK3BB7AU032766', 'KMHLB8ULDU069747', 'TMBAA25J883113476', 'TSMMZC21S00134117', 
#           'WDD1760082J557012', 'WVWZZZ3CZDE025430', 'YV1MW84F1B2624570']

### the calculation

Bellow you find the calculation, be aware that this is all written in python dataframes. And in this case with k=3 there are at least 2\*3 columns added to calculate the LOF for EACH car. So it will take quite some time for the whole process to finish. If you want to experiment with less intensive calculations then i suggest to change the amount of cars in the list. Also, it seems that the linux envirroment is noticeably faster, haven't look into why.


In [ ]:
everything_together(cars_list, 9)

Requesting query... ok.
Job ID: job_ECmSI9F73Y6wBfqNOV9tm5A1jn1w
Query running...
Query done.
Cache hit.

Retrieving results...
Got 14042 rows.

Total time taken 1.27 s.
Finished at 2017-12-02 13:30:36.
Empty DataFrame
Columns: [timestamp, value, vehicle, counter]
Index: []


/Users/Alexander_Baloche/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Requesting query... ok.
Job ID: job_1_ozfiRug843uylvTOAXc0BlBblK
Query running...
Query done.
Cache hit.

Retrieving results...
  Got page: 2; 80% done. Elapsed 9.89 s.
  Got page: 3; 100% done. Elapsed 12.79 s.
Got 248905 rows.

Total time taken 15.58 s.
Finished at 2017-12-02 13:30:57.
Empty DataFrame
Columns: [timestamp, value, vehicle, counter]
Index: []


### interesting results:
Now what is interesting is the difference between the histograms. For instance car4 of k=9 you see a classic staircase reduction. While if you look at car2 of k=9, you see that the average accelaration is much much higher (frequency tops somewhere around 12 kmh). And finally, looking at car5 of k=9 this person is almost exclusively accelarating at a much lower speed. This could indicate much more congestion driving, where you 'inch forward' most of the time. Looking at the data in this context I would think that the tires of car2 would be more 'worn' than for instance car5 which seems to go extremely cautionly. This is only looking at the data not at the state of the tires etc. 

The next step would be to physically measure the tire 'threading' over time and see if the parameter 'accelaration' is statistically sufficent to explain the rate of tire thread 'wearing'. 